In [10]:
import os
import sys

from models.model import FER2013CNN

In [11]:
#main imports

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchsummary import summary

from tqdm import tqdm
import cv2

In [12]:
from data.dataloader import load_dataloader

In [50]:
from utils.utils import line_graph

In [14]:
print(torch.backends.mps.is_available())

True


In [53]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [40]:
def train_model(model, train_loader, criterion, optimizer, epochs, val_loader=None):

    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model = model.to(device)
    
    train_accuracies = []
    val_accuracies = []

    train_losses = []
    val_losses = []
    
    for epoch in range(1, epochs + 1):
        model.train()  
        epoch_loss = 0.0
        
        correct, total = 0, 0
        
        with tqdm(train_loader, unit="batch", desc=f"Epoch {epoch}/{epochs}") as pbar:
            for images, labels in pbar:

                images = images.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()  
                outputs = model(images)  
                loss = criterion(outputs, labels)
                loss.backward()  
                optimizer.step()  

                epoch_loss += loss.item()
                _, prediction = torch.max(outputs, 1)
                correct += (prediction == labels).sum().item()
                total += labels.size(0)

                pbar.set_postfix(loss=epoch_loss / total, accuracy=100 * correct / total)

        if epoch % 5 == 0 and val_loader:
            model.eval()
            val_loss = 0.0
            val_correct = 0.0
            val_total = 0.0

            with torch.no_grad():     
                 with tqdm(val_loader, unit="batch", desc=f"Validation Round {epoch // 5}") as vpbar:
                    for images, labels in vpbar:
                        images = images.to(device)
                        labels = labels.to(device)
                        outputs = model(images)
                        loss = criterion(outputs, labels)
    
                        val_loss += loss.item()
                        _, prediction = torch.max(outputs, 1)
                        val_correct += (prediction == labels).sum().item()
                        val_total += labels.size(0)

                        val_acc = 100 * val_correct / val_total
                        vpbar.set_postfix(loss=val_loss / val_total, accuracy=val_acc)
            val_losses.append(val_loss)
            val_accuracies.append(val_acc)
        train_acc = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(train_acc)
        print(f"Epoch {epoch} : Train Accuracy = {train_acc:.2f}%, Loss = {epoch_loss:.4f}")
        print("-"*50)

    return model, train_accuracies, val_accuracies, train_losses, val_losses

In [41]:
cnn = FER2013CNN(num_classes=7)

In [42]:
criterion = nn.CrossEntropyLoss()

In [43]:
optimizer = optim.Adam(cnn.parameters(), lr=1e-3)

In [44]:
train_transform = transforms.Compose([
    transforms.ToTensor(),  
    transforms.RandomHorizontalFlip(),  
    transforms.RandomRotation(10),
    transforms.Normalize(mean=[0.5], std=[0.5])  
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [45]:
train_loader = load_dataloader("data/dataset/fer2013.csv", 'train', 32, transform=train_transform)
val_loader = load_dataloader("data/dataset/fer2013.csv", 'val', 32, transform=val_transform)
test_loader = load_dataloader("data/dataset/fer2013.csv", 'test', 32, transform=val_transform)

In [46]:
for images, labels in train_loader:
    assert images.size() == torch.Size([32, 1, 48, 48])
    assert labels.dtype == torch.int64
    print("test passed!")
    break

for images, labels in val_loader:
    assert images.size() == torch.Size([32, 1, 48, 48])
    assert labels.dtype == torch.int64
    print("test passed!")
    break

test passed!
test passed!


In [47]:
model, train_accuracies, val_accuracies, train_losses, val_losses = train_model(
    model=cnn,
    train_loader=train_loader,
    criterion=criterion,
    optimizer=optimizer,
    epochs=70,
    val_loader=val_loader
)

Epoch 1/70: 100%|███████████████████████████████| 898/898 [00:14<00:00, 62.04batch/s, accuracy=28.3, loss=0.0547]


Epoch 1 : Train Accuracy = 28.34%, Loss = 1571.6680
--------------------------------------------------


Epoch 2/70: 100%|███████████████████████████████| 898/898 [00:14<00:00, 63.99batch/s, accuracy=36.2, loss=0.0496]


Epoch 2 : Train Accuracy = 36.24%, Loss = 1424.5287
--------------------------------------------------


Epoch 3/70: 100%|███████████████████████████████| 898/898 [00:10<00:00, 88.88batch/s, accuracy=39.5, loss=0.0477]


...

Epoch 68 : Train Accuracy = 63.40%, Loss = 864.4036
--------------------------------------------------


Epoch 69/70: 100%|██████████████████████████████| 898/898 [00:11<00:00, 79.93batch/s, accuracy=64.2, loss=0.0297]


Epoch 69 : Train Accuracy = 64.18%, Loss = 852.2245
--------------------------------------------------


Validation Round 14: 100%|██████████████████████| 113/113 [00:00<00:00, 231.06batch/s, accuracy=60.7, loss=0.035]

Epoch 70 : Train Accuracy = 64.04%, Loss = 845.8452
--------------------------------------------------


In [51]:
line_graph(train_losses)

AttributeError: 'float' object has no attribute 'cpu'

<Figure size 1000x600 with 0 Axes>

In [54]:
from sklearn.metrics import classification_report

model.eval()  

y_true = []
y_pred = []

with torch.no_grad():  
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1) 

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.51      0.53       491
           1       0.00      0.00      0.00        55
           2       0.45      0.30      0.36       528
           3       0.83      0.85      0.84       879
           4       0.45      0.60      0.51       594
           5       0.80      0.72      0.76       416
           6       0.58      0.66      0.62       626

    accuracy                           0.62      3589
   macro avg       0.52      0.52      0.52      3589
weighted avg       0.61      0.62      0.61      3589



/Users/ravisaulog/cnn-project/cnnenv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ravisaulog/cnn-project/cnnenv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ravisaulog/cnn-project/cnnenv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

In [55]:
EMOTION_LABELS = {
    0: 'Angry',
    1: 'Disgust',
    2: 'Fear',
    3: 'Happy',
    4: 'Sad',
    5: 'Surprise',
    6: 'Neutral'
}

In [56]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)

EMOTION_LABELS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Webcam could not be opened.")
    exit()

try:
    while True:
        start_time = cv2.getTickCount()

        ret, frame = cap.read()
        if not ret:
            print("Could not read frame.")
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (48, 48))
            face_normalized = face_resized.astype('float32') / 255.0
            face_tensor = torch.tensor(face_normalized).unsqueeze(0).unsqueeze(0).float().to(device)
            assert face_tensor.size() == torch.Size([1, 1, 48, 48])

            model.eval()
            with torch.no_grad():
                outputs = model(face_tensor)
                softmax_outputs = F.softmax(outputs, dim=1)
                prob, predicted = torch.max(softmax_outputs, 1)
                emotion = EMOTION_LABELS[predicted.item()]
                
                


            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{emotion} : {prob.item() * 100 :.2f} %", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.6, (0, 255, 0), 2, cv2.LINE_AA)

        fps = cv2.getTickFrequency() / (cv2.getTickCount() - start_time)
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow("Emotion Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    cap.release()
    cv2.destroyAllWindows()

2025-05-21 14:51:56.026 python[30765:9090896] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-21 14:51:56.026 python[30765:9090896] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [1]:
model

NameError: name 'model' is not defined